In [ ]:
import pandas as pd
from collections import Counter
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the CSV file into a DataFrame
df = pd.read_csv('complete_dataset.csv')


print(f"\nDataset Shape: {df.shape}")
print(f"Total Games: {len(df)}")

# Display the first 5 rows
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Print the column names and their data types
print("\nColumn information:")
print(df.info())

print("\nDataset description:")
print(df.describe())

# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
}).sort_values('Missing Percentage', ascending=False)

print("\nMissing Values Analysis:")
print(missing_df[missing_df['Missing Count'] > 0])

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nNumber of Duplicate Rows: {duplicates}")

# Data type analysis
print("\nData Types:")
for col in df.columns:
    print(f"{col}: {df[col].dtype}")


BOARD GAME DATA ANALYSIS - ENHANCED VERSION

Dataset Shape: (7575, 23)
Total Games: 7575

First 5 rows of the dataset:
     id                                               name  year_published  \
0  2651                                         Power Grid            2004   
1  2511  Sherlock Holmes Consulting Detective: The Tham...            1981   
2  2655                                               Hive            2001   
3  2653                     Survive: Escape from Atlantis!            1982   
4  2346                                              DVONN            2001   

   min_players  max_players  playing_time  min_play_time  max_play_time  \
0            2            6           120            120            120   
1            1            8           120             60            120   
2            2            2            20             20             20   
3            2            4            60             45             60   
4            2            2          

In [2]:
# Identify numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numerical columns: {numerical_cols}")

# Basic statistics for numerical features
print("\nBasic Statistics for Numerical Features:")
for col in numerical_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Mean: {df[col].mean():.2f}")
        print(f"  Median: {df[col].median():.2f}")
        print(f"  Std: {df[col].std():.2f}")
        print(f"  Min: {df[col].min()}")
        print(f"  Max: {df[col].max()}")
        print(f"  Skewness: {df[col].skew():.2f}")
        print(f"  Kurtosis: {df[col].kurtosis():.2f}")

Numerical columns: ['id', 'year_published', 'min_players', 'max_players', 'playing_time', 'min_play_time', 'max_play_time', 'min_age', 'users_rated', 'average_rating', 'bayes_average', 'num_comments', 'num_weights', 'average_weight', 'bgg_rank']

Basic Statistics for Numerical Features:

id:
  Mean: 104770.21
  Median: 49050.00
  Std: 111332.49
  Min: 1002
  Max: 436217
  Skewness: 0.99
  Kurtosis: 0.01

year_published:
  Mean: 2001.48
  Median: 2009.00
  Std: 90.59
  Min: 0
  Max: 2025
  Skewness: -21.37
  Kurtosis: 469.19

min_players:
  Mean: 1.89
  Median: 2.00
  Std: 0.64
  Min: 0
  Max: 9
  Skewness: 1.46
  Kurtosis: 10.60

max_players:
  Mean: 4.61
  Median: 4.00
  Std: 6.92
  Min: 0
  Max: 120
  Skewness: 11.95
  Kurtosis: 159.89

playing_time:
  Mean: 141.87
  Median: 60.00
  Std: 792.43
  Min: 0
  Max: 60000
  Skewness: 58.60
  Kurtosis: 4314.87

min_play_time:
  Mean: 91.27
  Median: 45.00
  Std: 717.43
  Min: 0
  Max: 60000
  Skewness: 77.65
  Kurtosis: 6432.66

max_play_ti

In [3]:
# --- Outlier Detection ---
def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    if column not in df.columns or not pd.api.types.is_numeric_dtype(df[column]):
        return []
    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return outliers

# Detect outliers for key numerical columns
key_numerical_cols = ['average_rating', 'users_rated', 'playing_time', 'year_published']
for col in key_numerical_cols:
    if col in df.columns:
        outliers = detect_outliers_iqr(df, col)
        print(f"\n{col} outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}% of data)")
        if len(outliers) > 0:
            print(f"  Outlier range: {outliers.min()} to {outliers.max()}")


average_rating outliers: 102 (1.3% of data)
  Outlier range: 3.08434 to 9.27146

users_rated outliers: 984 (13.0% of data)
  Outlier range: 3012 to 131046

playing_time outliers: 630 (8.3% of data)
  Outlier range: 270 to 60000

year_published outliers: 464 (6.1% of data)
  Outlier range: 0 to 1978


In [4]:
# --- Categorical Feature Analysis ---
print("\n" + "="*50)
print("CATEGORICAL FEATURE ANALYSIS")
print("="*50)

# Function to extract and count items from a string column (fixed version)
def count_items(df, column_name):
    """Extract and count items from comma-separated string columns"""
    if column_name not in df.columns:
        print(f"Warning: Column '{column_name}' not found in dataset")
        return Counter()
    
    all_items = []
    for item_list in df[column_name].dropna():
        if pd.isna(item_list) or item_list == '':
            continue
        # Clean and split the string
        clean_list = str(item_list).strip()
        items = [item.strip() for item in clean_list.split(',') if item.strip()]
        all_items.extend(items)
    return Counter(all_items)

# Analyze categorical columns
categorical_columns = ['categories', 'mechanisms', 'designers', 'artists', 'publishers']
top_items = {}

for col in categorical_columns:
    if col in df.columns:
        item_counts = count_items(df, col)
        top_10 = item_counts.most_common(10)
        top_items[col] = top_10
        print(f"\nTop 10 {col.title()}:")
        for i, (item, count) in enumerate(top_10, 1):
            print(f"  {i}. {item}: {count}")
    else:
        print(f"\nColumn '{col}' not found in dataset")


CATEGORICAL FEATURE ANALYSIS

Top 10 Categories:
  1. Wargame: 2703
  2. Card Game: 1649
  3. Fantasy: 948
  4. World War II: 871
  5. Science Fiction: 711
  6. Fighting: 597
  7. Economic: 573
  8. Children's Game: 564
  9. Abstract Strategy: 549
  10. Animals: 546

Top 10 Mechanisms:
  1. Dice Rolling: 3013
  2. Hexagon Grid: 1872
  3. Hand Management: 1549
  4. Simulation: 1517
  5. Variable Player Powers: 1039
  6. Grid Movement: 1024
  7. Set Collection: 1004
  8. Open Drafting: 739
  9. Tile Placement: 737
  10. Modular Board: 732

Top 10 Designers:
  1. Reiner Knizia: 179
  2. (Uncredited): 123
  3. Joseph Miranda: 92
  4. Richard H. Berg: 88
  5. Jim Dunnigan: 85
  6. Frank Chadwick: 62
  7. Dean Essig: 58
  8. Bruno Cathala: 55
  9. Martin Wallace: 53
  10. Wolfgang Kramer: 47

Top 10 Artists:
  1. Rodger B. MacGowan: 318
  2. Redmond A. Simonsen: 251
  3. Mark Simonitch: 185
  4. Joe Youst: 162
  5. (Uncredited): 150
  6. Franz Vohwinkel: 136
  7. Charles Kibler: 105
  8. Mi

In [5]:
# --- Game Rating Analysis ---
if 'average_rating' in df.columns:
    # Rating distribution analysis
    print("Rating Distribution:")
    print(f"Games with rating >= 8.0: {len(df[df['average_rating'] >= 8.0])}")
    print(f"Games with rating >= 7.0: {len(df[df['average_rating'] >= 7.0])}")
    print(f"Games with rating < 6.0: {len(df[df['average_rating'] < 6.0])}")
    
    # Top rated games
    if 'name' in df.columns:
        print(f"\nTop 10 Highest Rated Games:")
        top_rated = df.nlargest(10, 'average_rating')[['name', 'average_rating', 'users_rated']]
        for idx, row in top_rated.iterrows():
            print(f"  {row['name']}: {row['average_rating']:.2f} ({row['users_rated']} ratings)")

# --- Temporal Analysis ---
if 'year_published' in df.columns:
    # Games by decade
    df['decade'] = (df['year_published'] // 10) * 10
    decade_counts = df.groupby('decade').size().sort_index()
    print("Games by Decade:")
    for decade, count in decade_counts.items():
        if not pd.isna(decade):
            print(f"  {int(decade)}s: {count} games")
    
    # Rating trends over time
    if 'average_rating' in df.columns:
        rating_by_decade = df.groupby('decade')['average_rating'].mean()
        print(f"\nAverage Rating by Decade:")
        for decade, rating in rating_by_decade.items():
            if not pd.isna(decade) and not pd.isna(rating):
                print(f"  {int(decade)}s: {rating:.2f}")


Rating Distribution:
Games with rating >= 8.0: 353
Games with rating >= 7.0: 2685
Games with rating < 6.0: 1360

Top 10 Highest Rated Games:
  Fields of Fire: Deluxe Edition: 9.27 (309 ratings)
  Die Festung Hamburg: 9.19 (50 ratings)
  Arydia: The Paths We Dare Tread: 9.18 (1904 ratings)
  On to Richmond II: The Union Strikes South: 9.15 (115 ratings)
  Global War 1939: 9.12 (39 ratings)
  Goblin Uprising: 9.11 (32 ratings)
  The Elder Scrolls: Betrayal of the Second Era: 9.07 (2094 ratings)
  The Elder Scrolls: Betrayal of the Second Era: 9.06 (2128 ratings)
  AFU: Armed Forces of Ukraine: 9.05 (516 ratings)
  OPPIDUM: 9.02 (35 ratings)
Games by Decade:
  0s: 15 games
  1800s: 1 games
  1810s: 2 games
  1840s: 1 games
  1850s: 5 games
  1860s: 2 games
  1870s: 1 games
  1880s: 4 games
  1890s: 5 games
  1900s: 3 games
  1910s: 4 games
  1920s: 5 games
  1930s: 10 games
  1940s: 11 games
  1950s: 16 games
  1960s: 49 games
  1970s: 409 games
  1980s: 574 games
  1990s: 670 games
  200

In [6]:
# Player count analysis
if 'min_players' in df.columns and 'max_players' in df.columns:
    df['player_range'] = df['max_players'] - df['min_players']
    print("Player Count Analysis:")
    print(f"  Average min players: {df['min_players'].mean():.1f}")
    print(f"  Average max players: {df['max_players'].mean():.1f}")
    print(f"  Average player range: {df['player_range'].mean():.1f}")
    
    # Most common player counts
    print(f"\nMost Common Player Ranges:")
    player_range_counts = df['player_range'].value_counts().head()
    for range_val, count in player_range_counts.items():
        print(f"  {int(range_val)} player range: {count} games")

# Complexity analysis (if weight data available)
if 'average_weight' in df.columns:
    print(f"\nComplexity Analysis:")
    print(f"  Average complexity: {df['average_weight'].mean():.2f}")
    
    # Complexity categories
    df['complexity_category'] = pd.cut(df['average_weight'], 
                                     bins=[0, 2, 3, 4, 5], 
                                     labels=['Light', 'Medium-Light', 'Medium-Heavy', 'Heavy'])
    complexity_counts = df['complexity_category'].value_counts()
    print("Complexity Distribution:")
    for category, count in complexity_counts.items():
        print(f"  {category}: {count} games")

# Most prolific publishers/designers
if 'publishers' in df.columns:
    publisher_counts = count_items(df, 'publishers')
    print("Most Prolific Publishers:")
    for pub, count in publisher_counts.most_common(5):
        print(f"  {pub}: {count} games")

if 'designers' in df.columns:
    designer_counts = count_items(df, 'designers')
    print(f"\nMost Prolific Designers:")
    for designer, count in designer_counts.most_common(5):
        print(f"  {designer}: {count} games")


Player Count Analysis:
  Average min players: 1.9
  Average max players: 4.6
  Average player range: 2.7

Most Common Player Ranges:
  0 player range: 2036 games
  2 player range: 1832 games
  3 player range: 1426 games
  4 player range: 865 games
  1 player range: 617 games

Complexity Analysis:
  Average complexity: 2.31
Complexity Distribution:
  Medium-Light: 3141 games
  Light: 2929 games
  Medium-Heavy: 1284 games
  Heavy: 213 games
Most Prolific Publishers:
  Inc.: 317 games
  Hobby Japan: 308 games
  999 Games: 293 games
  Ltd.: 280 games
  Korea Boardgames: 273 games

Most Prolific Designers:
  Reiner Knizia: 179 games
  (Uncredited): 123 games
  Joseph Miranda: 92 games
  Richard H. Berg: 88 games
  Jim Dunnigan: 85 games


In [7]:
print(f"Dataset Overview:")
print(f"  Total games analyzed: {len(df)}")
print(f"  Date range: {df['year_published'].min():.0f} - {df['year_published'].max():.0f}")

if 'average_rating' in df.columns:
    print(f"  Average game rating: {df['average_rating'].mean():.2f}")
    print(f"  Rating standard deviation: {df['average_rating'].std():.2f}")

if 'playing_time' in df.columns:
    print(f"  Average playing time: {df['playing_time'].mean():.0f} minutes")

if 'users_rated' in df.columns:
    print(f"  Total ratings in dataset: {df['users_rated'].sum():,}")
    print(f"  Average ratings per game: {df['users_rated'].mean():.0f}")


Dataset Overview:
  Total games analyzed: 7575
  Date range: 0 - 2025
  Average game rating: 6.69
  Rating standard deviation: 0.81
  Average playing time: 142 minutes
  Total ratings in dataset: 16,134,960
  Average ratings per game: 2130
